In [3]:
import numpy as np
from scipy.io import loadmat

In [11]:
# download the data from here: https://crcns.org/data-sets/bst/vta-1/about-vta-1 and copy the data folder here
d = loadmat('data/Saline_Task1_1.mat')
d.keys()

dict_keys(['__header__', '__version__', '__globals__', 'odor_onsets', 'reward_onsets', 'spiketimes', 'trial_type'])

In [12]:
d['spiketimes']

array([[4159025.4],
       [4159172.8],
       [4159325.1],
       ...,
       [6206275.2],
       [6206403.7],
       [6206555.7]], shape=(16561, 1))

In [ ]:

t_cs = d['odor_onsets'][0,0]

window_ms = (-150, 200)
bin_ms = 10
window_length_ms = window_ms[1] - window_ms[0]
assert(window_length_ms % bin_ms == 0)
n_bin = int(window_length_ms / bin_ms)
spike_counts = np.zeros((n_bin))

anchor_ms = t_cs
trial_start_ms = anchor_ms + window_ms[0]
trial_end_ms = anchor_ms + window_ms[1]

for ts in d['spiketimes'][:, 0]:
    if ts >= trial_start_ms and ts < trial_end_ms:
        spike_time_from_trial_start_ms = ts - trial_start_ms
        bin_idx = int(np.floor(spike_time_from_trial_start_ms / bin_ms))
        spike_counts[bin_idx] += 1
    elif ts >= trial_end_ms:
        break

spike_counts


5
13
21
28
30


array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
       0.])